# OSM data download

In [3]:
# This is an exapmle structure of a query we send to the Overpass API
"""
        ## this sets our output format
        [out:json]; 

        ## this defines our search area
        area(3600000000 + id of the land)->.searchArea; 

        ## parenthesis performs a union of the following retrieved data
        (
        node["key"="value"](area.searchArea);
        way["key"="value"](area.searchArea);
        relation["key"="value"](area.searchArea);
        );

        ## here we subset our union just to contain way(), node(), relation()
        way(r);

        ## this somehow gets the coordinates
        (._;>;); 

        ## important statement to set an output
        out geom; 
"""

'\n        ## this sets our output format\n        [out:json]; \n\n        ## this defines our search area\n        area(3600000000 + id of the land)->.searchArea; \n\n        ## parenthesis performs a union of the following retrieved data\n        (\n        node["key"="value"](area.searchArea);\n        way["key"="value"](area.searchArea);\n        relation["key"="value"](area.searchArea);\n        );\n\n        ## here we subset our union just to contain way(), node(), relation()\n        way(r);\n\n        ## this somehow gets the coordinates\n        (._;>;); \n\n        ## important statement to set an output\n        out geom; \n'

## Requests

In [4]:
import requests
import json

# This is the URL where our query is send to
overpass_url = "http://overpass-api.de/api/interpreter"

# This is our query we send to Overpass API for the stations: All stations also including subway stations
overpass_query_stations = """
    // set output to json file
    [out:json];
    // set search area to Portugal
    ( area[name="Portugal"]; )->.searchArea;
    // perform union with parenthesis
    (
      // AND statement by [key1=value1](and)[key2=value2]
    node["railway"="stop"]["subway"!="yes"]["station"!="subway"]["station"!="light_rail"]["light_rail"!="yes"](area.searchArea);
    node["railway"="station"]["subway"!="yes"]["station"!="subway"]["station"!="light_rail"]["light_rail"!="yes"](area.searchArea);
    node["public_transport"="stop_position"]["train"="yes"]["subway"!="yes"]["station"!="subway"]["station"!="light_rail"]["light_rail"!="yes"](area.searchArea);
    );
    (._;);
    out geom;
    """
# This is our query we send to Overpass API for the railways: 
# railway=rail or railway=narrow_guage, when tag usage then NOT industrial, military, tourism, freight
overpass_query_rails = """
    // set output to json file
    [out:json];
    // set search area to Portugal
    ( area[name="Portugal"]; )->.searchArea;
    // perform union with parenthesis (unionpart1; unionpart2)
    (
        // AND statement by [key1=value1](and)[key2=value2]
    way["railway"="narrow_gauge"]["usage"!="military"]["usage"!="industrial"]["usage"!="freight"]["usage"!="tourism"](area.searchArea);
    way["railway"="rail"]["usage"!="military"]["usage"!="industrial"]["usage"!="freight"]["usage"!="tourism"](area.searchArea);
    );
    (._;);
    out geom;
    """



## Download and check data

In [5]:
def get_data(query: str):
    """
    This function sends the query for the OSM Overpass API in string format and gives back a JSON response object
    Args:
        query: str = Query in string format of Overpass query language
    Returns: 
        A response data object in json format containing OSM data
    """
    response = requests.get(overpass_url, params={'data': query})
    return response.json()

railway rails

In [6]:
railway_rails = get_data(overpass_query_rails)
print('Type of railway rails:', type(railway_rails))
print('\nExample of one railway rail: \n',railway_rails['elements'][0])

Type of railway rails: <class 'dict'>

Example of one railway rail: 
 {'type': 'way', 'id': 4246983, 'bounds': {'minlat': 38.8880266, 'minlon': -9.0360783, 'maxlat': 38.8898715, 'maxlon': -9.0340947}, 'nodes': [1146585641, 25398728, 1146583996], 'geometry': [{'lat': 38.8898715, 'lon': -9.0340947}, {'lat': 38.8888339, 'lon': -9.0352147}, {'lat': 38.8880266, 'lon': -9.0360783}], 'tags': {'electrified': 'contact_line', 'frequency': '50', 'gauge': '1668', 'maxspeed': '190', 'maxspeed:source': 'Estimated at 190 km/h. It is in the category > 160 Km/h <= 220 Km/h according to https://www.infraestruturasdeportugal.pt/sites/default/files/files/files/diretorio_da_rede_2020_0.pdf', 'name': 'Linha 3 (Alverca)', 'passenger_lines': '4', 'railway': 'rail', 'railway:atc': 'yes', 'source': 'Bing', 'tracks': '1', 'usage': 'main', 'voltage': '25000'}}


railway stations

In [7]:
railway_stations = get_data(overpass_query_stations)
print('Type of railway stations:', type(railway_stations))
print('\nExample of one railway station: \n',railway_stations['elements'][0])

Type of railway stations: <class 'dict'>

Example of one railway station: 
 {'type': 'node', 'id': 24984214, 'lat': 38.609897, 'lon': -9.1015513, 'tags': {'local_ref': '2', 'name': 'Fogueteiro', 'public_transport': 'stop_position', 'railway': 'stop', 'train': 'yes', 'wikidata': 'Q8779789', 'wikipedia': 'pt:Estação Ferroviária de Fogueteiro'}}


## Save as json and geojson

In [8]:
import osm2geojson as o2g
import geojson
import os

def save_as_json_geojson(overpass_json, filename):
    """This function saves the overpass query results in folder original in json format in a json file and geojson file
    Arg:
        overpass_json = Json format of the Overpass result
        filename: str = Filename for the saving the files
    Return:
        .\original\filename.json
        .\original\filename.geojson
    """
    # Create a new folder .\original if it doesnÄt exist so far
    if os.path.exists("original") == False:
        os.mkdir("original")
    # Save as normal json file
    with open(f"original\\{filename}.json",mode="w") as file1:
        geojson.dump(overpass_json,file1)
    # Save as geojson file
    overpass_geojson = o2g.json2geojson(overpass_json) ## convert to a geojson
    with open(f"original\\{filename}.geojson",mode="w") as file2:
        geojson.dump(overpass_geojson,file2)
    return None

In [9]:
# Save railway_stations
save_as_json_geojson(railway_stations, 'railway_stations')

# Save railway_rails
save_as_json_geojson(railway_rails, 'railway_rails')